In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image

# Load MiDaS model
midas_model = torch.hub.load("intel-isl/MiDaS", "MiDaS")

# Set the model to evaluation mode
midas_model.eval()

# Load left and right images
left_image_path = 'Left/000001.png'
right_image_path = 'Right/000001.png'

# Load the left and right images using PIL
left_image = Image.open(left_image_path).convert('RGB')
right_image = Image.open(right_image_path).convert('RGB')

# Resize images to fit MiDaS model input size and normalize pixel values
resize = torch.nn.functional.interpolate
target_size = (384, 384)  # Target size for resizing

# Convert images to NumPy arrays and normalize pixel values
left_np = np.array(left_image).astype(np.float32) / 255.0
right_np = np.array(right_image).astype(np.float32) / 255.0

# Convert NumPy arrays to PyTorch tensors
left_tensor = torch.from_numpy(left_np).permute(2, 0, 1).unsqueeze(0)
right_tensor = torch.from_numpy(right_np).permute(2, 0, 1).unsqueeze(0)

# Resize images using torch.nn.functional.interpolate
left_resized = resize(left_tensor, size=target_size, mode='area')
right_resized = resize(right_tensor, size=target_size, mode='area')

# Perform depth estimation
with torch.no_grad():
    depth_prediction = midas_model(torch.cat([left_resized, right_resized], dim=0))

# Convert depth map tensor to numpy array

# Convert depth map to grayscale
# Extract the depth map from the model output
depth_map = depth_prediction[0].squeeze().cpu().numpy()

# Convert depth map to grayscale
depth_map_gray = (depth_map * 255).astype(np.uint8)

# Optionally, visualize or save the depth map
cv2.imwrite("depth_map.png", depth_map_gray)




In [ ]:
import cv2
import numpy as np
import os

# Load camera calibration data
def load_calib(calib_file):
    with open(calib_file, 'r') as f:
        lines = f.readlines()
        P0 = np.array([float(x) for x in lines[0].split(':')[1].strip().split()]).reshape(3, 4)
        P1 = np.array([float(x) for x in lines[1].split(':')[1].strip().split()]).reshape(3, 4)
    return P0, P1

# Paths to input images and calibration data
left_image_path = 'Left/000001.png'
right_image_path = 'Right/000001.png'
calib_file = 'Calib/000001.txt'

# Load camera calibration data
P0, P1 = load_calib(calib_file)

# Load left and right images
left_img = cv2.imread(left_image_path)
right_img = cv2.imread(right_image_path)

# Create a StereoSGBM object
stereo = cv2.StereoSGBM_create(numDisparities=16, blockSize=15)

# Compute the disparity map
disparity = stereo.compute(left_img, right_img)

# Reproject the disparity map to 3D space to obtain the depth map
depth_map = cv2.reprojectImageTo3D(disparity, P0)

# Save the depth map as a PNG file
cv2.imwrite('depth_image.png', depth_map)

In [ ]:
import cv2
import numpy as np
import os

# Load camera calibration data
def load_calib(calib_file):
    with open(calib_file, 'r') as f:
        lines = f.readlines()
        P0 = np.array([float(x) for x in lines[0].split(':')[1].strip().split()]).reshape(3, 4)
        P1 = np.array([float(x) for x in lines[1].split(':')[1].strip().split()]).reshape(3, 4)
        
        # Pad P0 with a bottom row of [0, 0, 0, 1] to make it 4x4
        P0 = np.vstack((P0, np.array([0, 0, 0, 1])))
    return P0, P1

# Paths to input images and calibration data
left_image_path = 'Project_Files/left/training/000792.png'
right_image_path = 'Project_Files/right/training/000792.png'
calib_file = 'Project_Files/calib/training/000792.txt'

# Load camera calibration data
P0, P1 = load_calib(calib_file)

# Load left and right images
left_img = cv2.imread(left_image_path)
right_img = cv2.imread(right_image_path)

# Create a StereoSGBM object
stereo = cv2.StereoSGBM_create(numDisparities=48, blockSize=1)

# Compute the disparity map
disparity = stereo.compute(left_img, right_img)

# Reproject the disparity map to 3D space to obtain the depth map
depth_map = cv2.reprojectImageTo3D(disparity, P0)

# Save the depth map as a PNG file
cv2.imwrite('depth_image_000792.png', depth_map)

# Process All the Files


pip install opencv-python


In [2]:
import cv2
import numpy as np
import os

# Load camera calibration data
def load_calib(calib_file):
    with open(calib_file, 'r') as f:
        lines = f.readlines()
        P0 = np.array([float(x) for x in lines[0].split(':')[1].strip().split()]).reshape(3, 4)
        P1 = np.array([float(x) for x in lines[1].split(':')[1].strip().split()]).reshape(3, 4)
        
        # Pad P0 with a bottom row of [0, 0, 0, 1] to make it 4x4
        P0 = np.vstack((P0, np.array([0, 0, 0, 1])))
    return P0, P1

def process_images(left_folder, right_folder, output_folder, calib_folder):
    # List all files in the left folder
    left_files = os.listdir(left_folder)

    for left_file in left_files:
        if left_file.endswith('.png'):
            # Form corresponding paths for left and right images and calibration data
            left_image_path = os.path.join(left_folder, left_file)
            right_image_path = os.path.join(right_folder, left_file)
            calib_file = os.path.join(calib_folder, left_file.replace('.png', '.txt'))

            # Load camera calibration data
            P0, P1 = load_calib(calib_file)

            # Load left and right images
            left_img = cv2.imread(left_image_path)
            right_img = cv2.imread(right_image_path)

            # Create a StereoSGBM object
            stereo = cv2.StereoSGBM_create(numDisparities=48, blockSize=1)

            # Compute the disparity map
            disparity = stereo.compute(left_img, right_img)

            # Reproject the disparity map to 3D space to obtain the depth map
            depth_map = cv2.reprojectImageTo3D(disparity, P0)

            # Save the depth map as a PNG file with the same name in the output folder
            output_file = os.path.join(output_folder, left_file.replace('.png', '_depth.png'))
            print("Saving depth map to:", output_file)
            success = cv2.imwrite(output_file, depth_map)
            if success:
                print("Depth map saved successfully.")
            else:
                print("Error: Unable to save depth map.")

# Paths for training and test datasets
train_left_folder = 'Project_Files/left/training'
train_right_folder = 'Project_Files/right/training'
train_output_folder = 'Project_Files/depth_images_local/training'
train_calib_folder = 'Project_Files/calib/training'

test_left_folder = 'Project_Files/left/testing'
test_right_folder = 'Project_Files/right/testing'
test_output_folder = 'Project_Files/depth_images_local/testing'
test_calib_folder = 'Project_Files/calib/testing'

# Process training images
process_images(train_left_folder, train_right_folder, train_output_folder, train_calib_folder)

# Process test images
process_images(test_left_folder, test_right_folder, test_output_folder, test_calib_folder)


Saving depth map to: Project_Files/depth_images_local/training/000792_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/training/000001_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/training/000000_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/training/000002_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/training/000003_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/testing/000792_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/testing/000001_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/testing/000000_depth.png
Depth map saved successfully.
Saving depth map to: Project_Files/depth_images_local/testing/000002_depth.png
Depth map saved successfully.
Saving depth m